In [ ]:
!pip install gdown

import gdown
import pandas as pd
import numpy as np

# Download both files
train_id = "1ZR4cnzoT4TA9uH8xeA4Pk_0jGZtBtciN"
test_id = "1Xt9wsLd2mWRONLjzT2wNdRIaFkVS_6q3"

gdown.download(f"https://drive.google.com/uc?id={train_id}", "UNSW_FEIIDS_train.csv", quiet=False)
gdown.download(f"https://drive.google.com/uc?id={test_id}", "UNSW_FEIIDS_test.csv", quiet=False)

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, Dense
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import time
import os


# Create folders for saving plots and models
PLOTS_FOLDER = "bilstm_plots1"
MODELS_FOLDER = "bilstm_models"
os.makedirs(PLOTS_FOLDER, exist_ok=True)
os.makedirs(MODELS_FOLDER, exist_ok=True)


# Detect runtime environment
def get_runtime_environment():
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.TPUStrategy(tpu)
        return "TPU", strategy
    except:
        gpus = tf.config.list_physical_devices('GPU')
        if gpus:
            return "GPU", tf.distribute.OneDeviceStrategy("/gpu:0")
        else:
            return "CPU", tf.distribute.OneDeviceStrategy("/cpu:0")


# Custom callback with minimal printing and full data collection
class TestEvaluationCallback(tf.keras.callbacks.Callback):
    def __init__(self, test_data, print_every=20):
        super().__init__()
        self.test_data = test_data
        self.test_loss = []
        self.test_accuracy = []
        self.print_every = print_every

    def on_epoch_end(self, epoch, logs=None):
        # Collect data EVERY epoch
        X_test, y_test = self.test_data
        test_loss, test_acc = self.model.evaluate(X_test, y_test, verbose=0)
        self.test_loss.append(test_loss)
        self.test_accuracy.append(test_acc)

        # Print only every N epochs
        if (epoch + 1) % self.print_every == 0 or epoch == 0:
            print(f"  Epoch {epoch+1:3d}/200 | Train Loss: {logs['loss']:.4f} | Train Acc: {logs['accuracy']:.4f} | Val Loss: {test_loss:.4f} | Val Acc: {test_acc:.4f}")


def plot_training_history(history, test_callback, environment, hidden_nodes, save_folder):
    """Plot and save training history for accuracy and loss using test set as validation"""
    epochs = range(1, len(history.history['accuracy']) + 1)

    # Plot Accuracy
    plt.figure(figsize=(10, 6))
    plt.plot(epochs, history.history['accuracy'], 'b-', label='Training Accuracy', linewidth=2)
    plt.plot(epochs, test_callback.test_accuracy, 'r-', label='Validation Accuracy', linewidth=2)
    plt.title(f'BiLSTM Training & Validation Accuracy\n{environment} - {hidden_nodes} Hidden Nodes', fontsize=14, fontweight='bold')
    plt.xlabel('Epoch', fontsize=12)
    plt.ylabel('Accuracy', fontsize=12)
    plt.legend(fontsize=11)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    acc_filename = os.path.join(save_folder, f"{environment}_{hidden_nodes}_acc.png")
    plt.savefig(acc_filename, dpi=150)
    plt.close()

    # Plot Loss
    plt.figure(figsize=(10, 6))
    plt.plot(epochs, history.history['loss'], 'b-', label='Training Loss', linewidth=2)
    plt.plot(epochs, test_callback.test_loss, 'r-', label='Validation Loss', linewidth=2)
    plt.title(f'BiLSTM Training & Validation Loss\n{environment} - {hidden_nodes} Hidden Nodes', fontsize=14, fontweight='bold')
    plt.xlabel('Epoch', fontsize=12)
    plt.ylabel('Loss', fontsize=12)
    plt.legend(fontsize=11)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    loss_filename = os.path.join(save_folder, f"{environment}_{hidden_nodes}_loss.png")
    plt.savefig(loss_filename, dpi=150)
    plt.close()


# =====================================================================
# CONFIGURATION - CHANGE THIS FOR EACH RUN
# =====================================================================
HIDDEN_NODES = 10  # Change to: 10, 20, 30, 40, 50, 60, 70, 80
# =====================================================================

ENVIRONMENT, strategy = get_runtime_environment()
print(f"\n{'='*70}")
print(f"Running on: {ENVIRONMENT}")
print(f"Hidden Nodes: {HIDDEN_NODES}")
print(f"{'='*70}\n")


# Load normalized data
train_df = pd.read_csv("UNSW_FEIIDS_train.csv")
test_df = pd.read_csv("UNSW_FEIIDS_test.csv")


# Identify and separate labels
label_cols = ['attack_cat', 'binary_label', 'Label', 'label']
existing_labels = [col for col in label_cols if col in train_df.columns]


X_full = train_df.drop(columns=existing_labels, errors='ignore').values
y_full = train_df['binary_label'].values if 'binary_label' in train_df.columns else train_df['label'].values
X_test_full = test_df.drop(columns=existing_labels, errors='ignore').values
y_test_full = test_df['binary_label'].values if 'binary_label' in test_df.columns else test_df['label'].values


print(f"Total train records: {len(X_full)}, Features: {X_full.shape[1]}")
print(f"Total test records: {len(X_test_full)}")


# Sample 14,000 for training and 3,500 for testing (as per Table 6)
np.random.seed(42)
train_indices = np.random.choice(len(X_full), 14000, replace=False)
test_indices = np.random.choice(len(X_test_full), 3500, replace=False)


# Create train and test sets
X_train = X_full[train_indices]
y_train = y_full[train_indices]
X_test = X_test_full[test_indices]
y_test = y_test_full[test_indices]


print(f"\nDataset splits:")
print(f"  Training: {X_train.shape[0]} samples")
print(f"  Testing: {X_test.shape[0]} samples")


# Reshape for LSTM: (samples, timesteps=1, features)
X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])


print(f"\nReshaped dimensions:")
print(f"  Train: {X_train.shape}")
print(f"  Test: {X_test.shape}\n")


# Configuration
EPOCHS = 200
BATCH_SIZE = 64
results_file = "bilstm_table_results.csv"


# Train single model
print(f"{'='*70}")
print(f"Training BiLSTM with {HIDDEN_NODES} hidden nodes on {ENVIRONMENT}")
print(f"{'='*70}\n")

with strategy.scope():
    # Build BiLSTM model (2 layers as per paper)
    model = Sequential([
        Bidirectional(LSTM(HIDDEN_NODES, return_sequences=True),
                     input_shape=(1, X_train.shape[2])),
        Bidirectional(LSTM(HIDDEN_NODES, return_sequences=False)),
        Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

print("Model built. Starting training (printing every 20 epochs)...\n")

# Create callback to track test performance (prints every 20 epochs)
test_callback = TestEvaluationCallback((X_test, y_test), print_every=20)

# Train model - SILENT MODE
start_train = time.time()
history = model.fit(
    X_train, y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose=0,  # SILENT - no epoch printing from fit()
    callbacks=[test_callback]  # Only our custom callback prints
)
train_time = time.time() - start_train

# Plot training history
print(f"\nTraining completed in {train_time:.2f}s")
print(f"Generating plots for {HIDDEN_NODES} hidden nodes...")
plot_training_history(history, test_callback, ENVIRONMENT, HIDDEN_NODES, PLOTS_FOLDER)
print("✓ Plots saved")

# Test model (final evaluation)
start_test = time.time()
y_pred_proba = model.predict(X_test, verbose=0)
test_time = time.time() - start_test

y_pred = (y_pred_proba > 0.5).astype(int).flatten()

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
test_loss = model.evaluate(X_test, y_test, verbose=0)[0]

# Store result
result = {
    'Environment': ENVIRONMENT,
    'Hidden_Nodes': HIDDEN_NODES,
    'Accuracy': round(accuracy, 4),
    'Train_Time': round(train_time, 2),
    'Test_Time': round(test_time, 2),
    'Test_Loss': round(test_loss, 4)
}

# =====================================================================
# SAVE MODEL
# =====================================================================
model_filename = os.path.join(MODELS_FOLDER, f"bilstm_{ENVIRONMENT}_{HIDDEN_NODES}nodes.keras")
model.save(model_filename)
print(f"\n✓ Model saved: {model_filename}")
# =====================================================================

# Save to CSV
result_df = pd.DataFrame([result])
if os.path.exists(results_file):
    existing_df = pd.read_csv(results_file)
    combined_df = pd.concat([existing_df, result_df], ignore_index=True)
    combined_df.to_csv(results_file, index=False)
else:
    result_df.to_csv(results_file, index=False)

print(f"\n{'='*70}")
print(f"RESULTS for {HIDDEN_NODES} hidden nodes:")
print(f"{'='*70}")
print(f"Accuracy: {accuracy*100:.2f}%")
print(f"Train Time: {train_time:.2f}s")
print(f"Test Time: {test_time:.2f}s")
print(f"Test Loss: {test_loss:.4f}")
print(f"\nSaved to: {results_file}")
print(f"Plots saved to: {PLOTS_FOLDER}/")
print(f"Model saved to: {MODELS_FOLDER}/")
print(f"{'='*70}\n")

# Display current results
if os.path.exists(results_file):
    final_results = pd.read_csv(results_file)
    print("All Results So Far:")
    print(final_results)